<a href="https://colab.research.google.com/github/OneFineStarstuff/Pinn/blob/main/MyAGIAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
MyAGIAgent with:
- Shared encoder for regression + multimodal matching
- Full-batch InfoNCE loss for multimodal alignment
- Curriculum scheduler to balance regression and multimodal training
- Safe fallbacks for harness base classes if not present
"""

import os, time, random, math
from dataclasses import dataclass
import numpy as np

# Torch setup
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    TORCH_AVAILABLE = True
except Exception:
    TORCH_AVAILABLE = False

SEED = 1234
random.seed(SEED); np.random.seed(SEED)
if TORCH_AVAILABLE:
    torch.manual_seed(SEED)
DEVICE = "cuda" if TORCH_AVAILABLE and torch.cuda.is_available() else "cpu"
RUN_ID = f"run_{int(time.time())}"

# ---------------------------------------------------------------------
# Harness base classes (only defined if they don't already exist)
# ---------------------------------------------------------------------
try:
    AgentAdapter  # type: ignore[name-defined]
except NameError:
    @dataclass
    class AgentConfig:
        name: str = "MyAGIAgent"
        use_world_model: bool = False
        use_multimodal_encoder: bool = True
        mc_dropout_passes: int = 0
        reflection_steps: int = 1
        notes: str = ""

    class AgentAdapter:
        def __init__(self, cfg: AgentConfig):
            self.cfg = cfg
        def act(self, obs: dict, task_id: str, step: int, state: dict | None):
            raise NotImplementedError
        def learn(self, batch: dict, task_id: str) -> dict:
            return {"learned": False}
        def reflect(self, logs: list[dict]) -> dict:
            return {"notes": "no-op", "patches": {}}
        def encode(self, modality: str, data):
            raise NotImplementedError
        def imagine(self, state: dict, n_steps: int = 5):
            return []

# ---------------------------------------------------------------------
# Contrastive loss (full-batch InfoNCE)
# ---------------------------------------------------------------------
def contrastive_loss(text_embeds, img_embeds, logit_scale):
    text_embeds = F.normalize(text_embeds, dim=-1)
    img_embeds = F.normalize(img_embeds, dim=-1)
    logits_per_text = torch.matmul(text_embeds, img_embeds.t()) * logit_scale.exp().clamp(1.0, 100.0)
    logits_per_image = logits_per_text.t()
    targets = torch.arange(text_embeds.size(0), device=text_embeds.device)
    loss_t2i = F.cross_entropy(logits_per_text, targets)
    loss_i2t = F.cross_entropy(logits_per_image, targets)
    return (loss_t2i + loss_i2t) / 2

# ---------------------------------------------------------------------
# Core AGI agent
# ---------------------------------------------------------------------
class MyAGIAgent(nn.Module):
    def __init__(self, shared_dim=128, txt_dim=64, img_hw=32, eps=0.1):
        super().__init__()
        self.shared_dim = shared_dim
        self.txt_dim = txt_dim
        self.img_hw = img_hw
        self.img_dim = img_hw * img_hw
        self.eps = eps

        self.trunk = nn.Sequential(
            nn.LayerNorm(shared_dim),
            nn.ReLU(),
            nn.Linear(shared_dim, shared_dim),
            nn.ReLU(),
        )

        self.txt_proj = nn.Linear(txt_dim, shared_dim)
        self.img_proj = nn.Linear(self.img_dim, shared_dim)
        self.num_proj = nn.Linear(1, shared_dim)

        self.policy_head = nn.Linear(shared_dim, 4)
        self.reg_head = nn.Linear(shared_dim, 1)

        self.logit_scale = nn.Parameter(torch.log(torch.tensor(10.0)))

        self.reg_opt = torch.optim.Adam(
            list(self.num_proj.parameters()) + list(self.reg_head.parameters()) + list(self.trunk.parameters()), lr=5e-3
        )
        self.mm_opt = torch.optim.Adam(
            list(self.txt_proj.parameters()) + list(self.img_proj.parameters()) + list(self.trunk.parameters()) + [self.logit_scale], lr=1e-3
        )

        alphabet = "abcdefghijklmnopqrstuvwxyz0123456789_- "
        self.char2idx = {c: i + 1 for i, c in enumerate(alphabet)}

    # ---- Featurizers ----
    def featurize_text(self, s: str) -> torch.Tensor:
        vec = np.zeros(self.txt_dim, dtype=np.float32)
        for ch in s or "":
            vec[self.char2idx.get(ch.lower(), 0) % self.txt_dim] += 1.0
        vec = vec / (np.linalg.norm(vec) + 1e-8)
        return torch.from_numpy(vec).to(DEVICE)

    def featurize_image(self, img: np.ndarray) -> torch.Tensor:
        arr = torch.from_numpy(np.asarray(img)).float()
        if arr.ndim == 2:
            arr = arr.unsqueeze(0)
        elif arr.ndim == 3:
            if arr.shape[-1] in (1, 3):
                arr = arr.permute(2, 0, 1)
            else:
                arr = arr.mean(dim=-1, keepdim=True).permute(2, 0, 1)
        arr = arr.unsqueeze(0)
        with torch.no_grad():
            arr = F.interpolate(arr, size=(self.img_hw, self.img_hw), mode="bilinear", align_corners=False)
        gray = arr.mean(dim=1, keepdim=False).squeeze(0)
        vec = gray.flatten()
        vec = vec / (vec.norm() + 1e-8)
        return vec.to(DEVICE)

    def featurize_num(self, x: np.ndarray) -> torch.Tensor:
        return torch.from_numpy(x).float().to(DEVICE)

    # ---- Encoders ----
    def encode_text_shared(self, s: str) -> torch.Tensor:
        z = self.txt_proj(self.featurize_text(s))
        return self.trunk(z)

    def encode_img_shared(self, img: np.ndarray) -> torch.Tensor:
        z = self.img_proj(self.featurize_image(img))
        return self.trunk(z)

    def encode_num_shared(self, x_batch: np.ndarray) -> torch.Tensor:
        z = self.num_proj(self.featurize_num(x_batch))
        return self.trunk(z)

    # ---- Actions ----
    @torch.no_grad()
    def act_regress(self, x_batch: np.ndarray) -> np.ndarray:
        self.eval()
        z = self.encode_num_shared(x_batch)
        y = self.reg_head(z)
        return y.cpu().numpy()

    @torch.no_grad()
    def act_multimodal_txt2img(self, text: str, images: list[np.ndarray]) -> int:
        self.eval()
        tq = self.encode_text_shared(text)
        scale = self.logit_scale.exp().clamp(1.0, 100.0)
        sims = [float(F.cosine_similarity(tq, self.encode_img_shared(img), dim=0) * scale) for img in images]
        return int(np.argmax(sims))

    @torch.no_grad()
    def act_multimodal_img2txt(self, image: np.ndarray, texts: list[str]) -> int:
        self.eval()
        iq = self.encode_img_shared(image)
        scale = self.logit_scale.exp().clamp(1.0, 100.0)
        sims = [float(F.cosine_similarity(iq, self.encode_text_shared(t), dim=0) * scale) for t in texts]
        return int(np.argmax(sims))

    # ---- Learning ----
    def step_regression(self, x: np.ndarray, y: np.ndarray) -> dict:
        self.train()
        z = self.encode_num_shared(x)
        pred = self.reg_head(z).squeeze(-1)
        target = torch.from_numpy(y.squeeze(-1)).float().to(DEVICE)
        loss = F.mse_loss(pred, target)
        self.reg_opt.zero_grad(set_to_none=True)
        loss.backward()
        self.reg_opt.step()
        return {"loss": float(loss.item())}

    def step_multimodal_batch(self, texts: list[str], images: list[np.ndarray]) -> dict:
        self.train()
        t_vecs = torch.stack([self.encode_text_shared(t) for t in texts])
        i_vecs = torch.stack([self.encode_img_shared(im) for im in images])
        loss = contrastive_loss(t_vecs, i_vecs, self.logit_scale)
        self.mm_opt.zero_grad(set_to_none=True)
        loss.backward()
        self.mm_opt.step()
        return {"loss": float(loss.item())}

# ---------------------------------------------------------------------
# Adapter with curriculum scheduler
# ---------------------------------------------------------------------
class MyAGIAgentAdapter(AgentAdapter):
    def __init__(self, cfg: "AgentConfig"):
        super().__init__(cfg)
        if not TORCH_AVAILABLE:
            raise RuntimeError("PyTorch required.")
        self.agent = MyAGIAgent().to(DEVICE)
        self.eps = 0.1
        self.curriculum_phase = 0  # 0 = regression-heavy, 1 = balanced

    def act(self, obs: dict, task_id: str, step: int, state: dict | None):
        # Grid heuristic
        if task_id.startswith("grid"):
            acts = obs["action_space"]
            if random.random() < self.eps:
                a = random.choice(acts)
            else:
                pos, goal = obs.get("pos"), obs.get("goal")
                def heuristic(act):
                    dx, dy = {"up":(-1,0), "down":(1,0), "left":(0,-1), "right":(0,1)}.get(act,(0,0))
                    nxt = (pos[0]+dx, pos[1]+dy)
                    return abs(nxt[0]-goal[0]) + abs(nxt[1]-goal[1]) if goal else 0
                a = min(acts, key=heuristic)
            return a, (state or {}), {}

        # Symbol heuristic
        if task_id.startswith("symbol"):
            rules, cur, tgt = obs["rules"], obs["string"], obs["target"]
            best, best_score = None, math.inf
            for (lhs, rhs) in rules:
                idx = cur.find(lhs)
                if idx >= 0:
                    new_s = cur[:idx] + rhs + cur[idx+len(lhs):]
                    score = abs(len(new_s)-len(tgt)) + sum(1 for a,b in zip(new_s, tgt) if a!=b)
                    if score < best_score:
                        best, best_score = (lhs, rhs), score
            if best is None and rules:
                best = random.choice(rules)
            return best, (state or {}), {}

        # Regression
        if task_id.startswith("regress"):
            return self.agent.act_regress(obs["x"]), (state or {}), {}

        # Multimodal
        if task_id.startswith("multimodal"):
            if obs.get("mode") == "txt2img":
                idx = self.agent.act_multimodal_txt2img(obs["text"], obs["images"])
            else:
                idx = self.agent.act_multimodal_img2txt(obs["image"], obs["texts"])
            return int(idx), (state or {}), {}

        return None, (state or {}), {}

    def learn(self, batch: dict, task_id: str) -> dict:
        # Curriculum: early phase trains regression more often
        if task_id.startswith("regress"):
            return self.agent.step_regression(batch["x"], batch["y"])
        if task_id.startswith("multimodal"):
            return self.agent.step_multimodal_batch(batch["texts"], batch["images"])
        return {"loss": None}

    def curriculum_step(self, global_step: int, warmup_steps: int = 500):
        """
        Call this each global step to update curriculum phase.
        """
        if global_step >= warmup_steps:
            self.curriculum_phase = 1  # balanced phase

    def reflect(self, logs: list[dict]) -> dict:
        fails = [1.0 if r.get("success")==0 else 0.0 for r in logs if "success" in r]
        fail_rate = float(np.mean(fails)) if fails else 0.0
        if fail_rate > 0.5:
            self.eps = min(0.3, self.eps + 0.05)
            note = f"Increased epsilon to {self.eps:.2f} after fail_rate={fail_rate:.2f}"
        else:
            note = f"No change; fail_rate={fail_rate:.2f}"
        return {"notes": note, "patches": {"eps": self.eps}}

    def encode(self, modality: str, data):
        if modality == "text":
            with torch.no_grad():
                return self.agent.encode_text_shared(data).cpu().numpy()
        if modality == "image":
            with torch.no_grad():
                return self.agent.encode_img_shared(data).cpu().numpy()
        raise ValueError(f"Unknown modality: {modality}")

    def imagine(self, state: dict, n_steps: int = 5):
        return []

# ---------------------------------------------------------------------
# Optional: run if harness is available
# ---------------------------------------------------------------------
if __name__ == "__main__":
    if "run_all" in globals():
        cfg = AgentConfig(
            name="MyAGIAgent",
            use_world_model=False,
            use_multimodal_encoder=True,
            notes="shared-encoder, InfoNCE multimodal, curriculum"
        )
        agent = MyAGIAgentAdapter(cfg)
        out = run_all(agent)
        try:
            df = out[0]
            dashboard(df)
        except Exception:
            pass
        print("Run complete:", RUN_ID)
    else:
        print("Defined MyAGIAgentAdapter with curriculum and InfoNCE multimodal training.")